In [ ]:
import os
import pandas as pd
import numpy as np
import datetime
import tensorflow as tf
import tqdm

In [2]:
TF_RECORDS = "./data/tf_records/"

In [13]:
def _parse_line(line):
    splits = line.strip().split(" ")
    l = splits[0], 
    g = splits[1].split(":")[1]
    f = [split.split(":")[1] for split in splits[2:]]

    return int(g), np.array(f).astype(np.float32), int(l[0])

In [14]:
data_path = "/media/akanyaani/Disk2/Ml_Data/LTR/MSLR-WEB10K/Fold1/test.txt"

In [15]:
def create_int_feature(values): 
    feature = tf.train.Feature(int64_list=tf.train.Int64List(value=list(values))) 
    return feature 

def create_float_feature(values): 
    feature = tf.train.Feature(float_list=tf.train.FloatList(value=list(values))) 
    return feature

def serialize_example(q_id, feature, labels):
    feature = {
        'q_id': create_int_feature(q_id),
        'feature': create_float_feature(feature),
        'labels': create_int_feature(labels)
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

In [16]:
def create_tf_records(data_path, per_file_limit=50000):
    print("Creating TF Records...............")
    
    filename = TF_RECORDS + str(datetime.datetime.now().timestamp()) + ".tfrecord"
    tf_writer = tf.io.TFRecordWriter(filename)
    doc_counts = 0
    with open(data_path, 'r') as f:
        for line in tqdm.tqdm(f):
            g, f, l = _parse_line(line)
            
            print(g)
            print(f)
            print(l)
            example = serialize_example([g], f, [l])
            tf_writer.write(example)
            doc_counts += 1
            if doc_counts >= per_file_limit:
                tf_writer.write(example)
                doc_counts = 0
                tf_writer.close()
                filename = TF_RECORDS + str(datetime.datetime.now().timestamp()) + ".tfrecord"
                tf_writer = tf.io.TFRecordWriter(filename)
    tf_writer.close()

In [ ]:
create_tf_records(data_path)